<h1>1.读取认证信息和租户信息</h1>
认证信息不需要在每次调用接口时都去通过登录重新获取，认证信息的有效时间是7天，因此在7天内都可以重复使用，可以直接读取user_info.json中保存的认证信息。

In [ ]:
import json

with open('user_info.json', 'r', encoding='utf-8') as f:
    payload = json.load(f)

token=payload["token"]
tenantId=payload["tenantId"]
print("认证信息：",token)
print("租户信息：",tenantId)

<h1>2.通过Http推送实测数据</h1>
接口参数是数组，可以同时推送多个设备的实测数据，每个设备需要设置deviceId、telemetryDataJson两个参数信息。</br>
deviceId是导入设备时自动生成的一个设备Id，需要通过设备信息接口获取。</br>
telemetryDataJson是单个时间戳的多个指标数据，telemetryDataJson 是一个 json 对象的序列化字符串，ts是时间戳（毫秒），value 是一个 kv 字典，key 表示设备的指标名称，如：Discharge、WaterLevel 等，value 表示指标数据的实测值</br>

In [ ]:
import os
from dhicn_iot_service import *
import time

configuration=Configuration.get_default_copy()
configuration.host="https://online-products.dhichina.cn/gateway/iot-service" #服务地址
api_client=ApiClient(configuration)
api_client.set_default_header('Authorization',token)
api_client.set_default_header('tenantId',tenantId)

#获取设备信息
device_instance=DeviceApi(api_client)
param={
    'page_index':1,
    'page_size':1000
}
response = device_instance.api_v1_iot_device_list_get(**param)

#构建参数
values={
    'Discharge':160,
    'WaterLevel':100
}
telemetryDataJson={
    'ts':time.time()*1000,
    'values':values
}
telemetry_data_json=f'{telemetryDataJson}'
param=[{"deviceId":response["data"]["list"][0]["id"],"telemetryDataJson":telemetry_data_json},{"deviceId":response["data"]["list"][1]["id"],"telemetryDataJson":telemetry_data_json}]
print(param)

#接口调用
telemetry_instance=TelemetryApi(api_client)
response = telemetry_instance.api_v1_iot_save_telemetry_data_batch_post(save_telemetry_data_input=param)
print("result:", response)
if response["status"] == False:
    os._exit()

<h1>3.获取实测数据</h1>
推送实测数据有三种方式，包括http、opcua、mqtt，分别对应了不同的推送方式，但获取数据的接口都是同一个。也就是如下所示接口</br>
在获取数据时需要用到设备ID/指标名称，因此需要先获取设备信息，解析出deviceId和Indicator。

In [ ]:
from dhicn_iot_service import *

def find_key(key, array):
    if isinstance(array, dict):
        if array['key']==key:
            return array['value']
        else:
             return None
    elif isinstance(array, list):
        for item in array:
            result = find_key(key, item)
            if result is not None:
                return result


configuration=Configuration.get_default_copy()
configuration.host="https://online-products.dhichina.cn/gateway/iot-service" #服务地址
api_client=ApiClient(configuration)
api_client.set_default_header('Authorization',token)
api_client.set_default_header('tenantId',tenantId)

#获取设备信息
device_instance=DeviceApi(api_client)
param={
    'page_index':1,
    'page_size':1000
}
response = device_instance.api_v1_iot_device_list_get(**param)
attributes=response["data"]["list"][0]["attributes"]
indicators = find_key("indicators",attributes)

print(indicators)
indicators=json.loads(indicators)
print(indicators[0]["Indicator"])
print(indicators[1]["Indicator"])

#构建参数
timeseries_input=[{
        "deviceId": response["data"]["list"][0]["id"],
        "endTs": "2023-08-04T18:13:27",
        "keys": [indicators[0]["Indicator"],indicators[0]["Indicator"]],
        "startTs": "2023-08-03T16:13:27",
        "orderBy": "asc"
    },
    {
        "deviceId": response["data"]["list"][1]["id"],
        "endTs": "2023-08-04T18:13:27",
        "keys": [indicators[0]["Indicator"],indicators[0]["Indicator"]],
        "startTs": "2023-08-03T16:13:27",
        "orderBy": "asc"
    }
]
telemetry_instance=TelemetryApi(api_client)
response=telemetry_instance.api_v2_iot_timeseries_batch_post(timeseries_input=timeseries_input)
print("result:", response)
if response["status"] == False:
    os._exit()